# Import Libraries

In [ ]:
from IPython.display import clear_output
!pip3 uninstall -y sklearn
!pip3 install lazypredict
!pip3 install flaml
!pip3 install autogluon
clear_output()

In [ ]:
import sklearn

import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from autogluon.tabular import TabularDataset, TabularPredictor



from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split


from lightgbm import LGBMClassifier
from flaml import AutoML
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install geopy Nominatim folium eli5 pdpbox colorama
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
import folium
import math
import plotly.graph_objects as go
import plotly.express as px
import eli5
import graphviz
import networkx as nx


from eli5.sklearn import PermutationImportance
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import tree
from matplotlib import pyplot as plt
from string import ascii_letters
from colorama import Fore, Back, Style
y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA

import tensorflow as tf
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.feature_selection import mutual_info_classif, SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
automl = AutoML()

# Load Dataset

In [ ]:
df = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv('../input/spaceship-titanic/test.csv')
submission = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')

In [ ]:
df.head(10)

# Extracting data information

## Description

#### Train

In [ ]:
df.info()

#### Test

In [ ]:
test.info()

## Feature identification

#### Train

In [ ]:
df.columns

#### Test

In [ ]:
test.columns

## Data numeric

#### Train

In [ ]:
numeric=['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num=df.select_dtypes(include=numeric)
df_num.columns

#### Test

In [ ]:
numeric=['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num=test.select_dtypes(include=numeric)
df_num.columns

## Data categorical

#### Train

In [ ]:
df_cat=df.select_dtypes(include='object')
df_cat.head(3)

#### Test

In [ ]:
df_cat=test.select_dtypes(include='object')
df_cat.head(3)

# Drop Columns

## Reason
These columns won't provide any importance to models as they are purely categorical 

#### Train

In [ ]:
df = df.drop(['PassengerId','Cabin', 'Name'], axis=1)

#### Test

In [ ]:
test = test.drop(['PassengerId','Cabin', 'Name'], axis=1)

## Missing Value

#### Train

In [ ]:
null=pd.DataFrame(df.isnull().sum(),columns=["Null Values"])
null["% Missing Values"]=(df.isna().sum()/len(df)*100)
null = null[null["% Missing Values"] > 0]
null.style.background_gradient(cmap='viridis',low =0.2,high=0.1) 

In [ ]:
null_cat=null.select_dtypes(include='object')
null_cat.head(3)

In [ ]:
imp_mean_cat = SimpleImputer(strategy='most_frequent')
imp_mean_cat.fit(df[['HomePlanet', 'CryoSleep', 'Destination']])
df[['HomePlanet', 'CryoSleep', 'Destination']] = imp_mean_cat.transform(df[['HomePlanet', 'CryoSleep', 'Destination']])

In [ ]:
numeric=['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
null_num=null.select_dtypes(include=numeric)
null_num.head(5)

In [ ]:
imp_mean_num = SimpleImputer(strategy='median')
imp_mean_num.fit(df[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'VIP']])
df[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'VIP']] = imp_mean_num.transform(df[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'VIP']])

In [ ]:
null=pd.DataFrame(df.isnull().sum(),columns=["Null Values"])
null["% Missing Values"]=(df.isna().sum()/len(df)*100)
null = null[null["% Missing Values"] > 0]
null.style.background_gradient(cmap='viridis',low =0.2,high=0.1) 

#### Test

In [ ]:
null=pd.DataFrame(test.isnull().sum(),columns=["Null Values"])
null["% Missing Values"]=(df.isna().sum()/len(df)*100)
null = null[null["% Missing Values"] > 0]
null.style.background_gradient(cmap='viridis',low =0.2,high=0.1) 

# Exploratory Data Analysis

## Numerical Approach

### Statistical Summary

#### Train

In [ ]:
describeNum = df.describe(include =['float64', 'int64', 'float', 'int'])
describeNum.T.style.background_gradient(cmap='viridis',low=0.2,high=0.1)

In [ ]:
describeNumCat = df.describe(include=["O"])
describeNumCat.T.style.background_gradient(cmap='viridis',low=0.2,high=0.1)

#### Test

In [ ]:
describeNum = test.describe(include =['float64', 'int64', 'float', 'int'])
describeNum.T.style.background_gradient(cmap='viridis',low=0.2,high=0.1)

In [ ]:
describeNumCat = test.describe(include=["O"])
describeNumCat.T.style.background_gradient(cmap='viridis',low=0.2,high=0.1)

## Graphic Approach

### Correlation heatmap

Now how to correlate between data variables.

Correlation is represented as a value between -1 and +1 where +1 indicates the highest positive correlation, -1 indicates the highest negative correlation, and 0 indicates no correlation.

#### Train

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(30,35))
sns.heatmap(df.corr(),  cmap='coolwarm', vmax=.3, center=0,
            square=True, linewidths=.5,annot=True)
plt.show()

In [ ]:
corr=df.corr()
indices = corr.index.values
cor_matrix = np.asmatrix(corr)
G = nx.from_numpy_matrix(cor_matrix)
G = nx.relabel_nodes(G,lambda x: indices[x])
G.edges(data=True)

In [ ]:
def corr_network(G, corr_direction, min_correlation):
    H = G.copy()

    for s1, s2, weight in G.edges(data=True):       
        if corr_direction == "positive":
            if (weight["weight"] < 0 or weight["weight"] < min_correlation) or weight["weight"]==1:
                H.remove_edge(s1, s2)
        else:
            if (weight["weight"] >= 0 or weight["weight"] > min_correlation) or weight["weight"]==1:
                H.remove_edge(s1, s2)
                
    edges,weights = zip(*nx.get_edge_attributes(H,'weight').items())

    weights = tuple([(1+abs(x))**2 for x in weights])
   
    d = dict(nx.degree(H))
    nodelist=d.keys()
    node_sizes=d.values()
    
    positions=nx.circular_layout(H)
    
    plt.figure(figsize=(15,15))

    nx.draw_networkx_nodes(H,positions,node_color='#d100d1',nodelist=nodelist,
                       node_size=tuple([x**3 for x in node_sizes]),alpha=0.8)

    nx.draw_networkx_labels(H, positions, font_size=14)

    if corr_direction == "positive":
        edge_colour = plt.cm.summer 
    else:
        edge_colour = plt.cm.autumn
        
    nx.draw_networkx_edges(H, positions, edgelist=edges,style='solid',
                          width=weights, edge_color = weights, edge_cmap = edge_colour,
                          edge_vmin = min(weights), edge_vmax=max(weights))
    plt.axis('off')
    plt.show()

In [ ]:
corr_network(G, corr_direction="positive",min_correlation = 0)

In [ ]:
corr_network(G, corr_direction="negative",min_correlation = -0.2)

#### Test

In [ ]:
test.corr()

In [ ]:
plt.figure(figsize=(30,35))
sns.heatmap(test.corr(),  cmap='coolwarm', vmax=.3, center=0,
            square=True, linewidths=.5,annot=True)
plt.show()

In [ ]:
corr=test.corr()
indices = corr.index.values
cor_matrix = np.asmatrix(corr)
G = nx.from_numpy_matrix(cor_matrix)
G = nx.relabel_nodes(G,lambda x: indices[x])
G.edges(data=True)

In [ ]:
corr_network(G, corr_direction="positive",min_correlation = 0)

In [ ]:
corr_network(G, corr_direction="negative",min_correlation = 0)

# Box Plots

#### Train

In [ ]:
numeric=['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num=df.select_dtypes(include=numeric)
df_num.columns
featuresNum = df_num.columns

plt.figure(figsize=(50, 40))
for i in range(0, len(featuresNum)):
    plt.subplot(1, len(featuresNum), i+1)
    sns.boxplot(y=df[featuresNum[i]], color='green', orient='v')

#### Test

In [ ]:
numeric=['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num=test.select_dtypes(include=numeric)
df_num.columns
featuresNum = df_num.columns

plt.figure(figsize=(50, 40))
for i in range(0, len(featuresNum)):
    plt.subplot(1, len(featuresNum), i+1)
    sns.boxplot(y=df[featuresNum[i]], color='green', orient='v')

### Transported count

In [ ]:
df_nottransported = df[(df["Transported"] == 0)]
df_nottransported.shape

In [ ]:
df_transported = df[(df["Transported"] == 1)]
df_transported.shape

In [ ]:
value_counts = df["Transported"].value_counts().to_dict()
fig, ax = plt.subplots()
_ = ax.pie(x=[value_counts[False], value_counts[True]], labels=['Not transported', 'transported'], colors=['#003f5c', '#ffa600'], textprops={'color': '#040204'})
_ = ax.axis('equal')
_ = ax.set_title('Transported')

### How transportation is related to home planet

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

_ = sns.countplot(x="HomePlanet", hue="Transported", data=df, palette="nipy_spectral",
              order=df['HomePlanet'].value_counts().index)

_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
_ = ax.set(xlabel="HomePlanet", ylabel="Counts")
plt.legend(bbox_to_anchor=(0.945, 0.90), labels=['Not Transported', 'Transported'])

### How transportation is related to cyrosleep?

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

_ = sns.countplot(x="CryoSleep", hue="Transported", data=df, palette="nipy_spectral",
              order=df['CryoSleep'].value_counts().index)

_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
_ = ax.set(xlabel="CryoSleep", ylabel="Counts")
plt.legend(bbox_to_anchor=(0.945, 0.90), labels=['Not Transported', 'Transported'])

### How transportation is related to destination?

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

_ = sns.countplot(x="Destination", hue="Transported", data=df, palette="nipy_spectral",
              order=df['Destination'].value_counts().index)

_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
_ = ax.set(xlabel="Destination", ylabel="Counts")
plt.legend(bbox_to_anchor=(0.945, 0.90), labels=['Not Transported', 'Transported'])

### How transportation is related to VIP?

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

_ = sns.countplot(x="VIP", hue="Transported", data=df, palette="nipy_spectral",
              order=df['VIP'].value_counts().index)

_ = ax.set_xticklabels(['False','True'], rotation=90)
_ = ax.set(xlabel="VIP", ylabel="Counts")
plt.legend(bbox_to_anchor=(0.945, 0.90), labels=['Not Transported', 'Transported'])

### How transportation is related with age?

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(10,20),sharex=False)

df_age_transported=pd.DataFrame(df.loc[(df['Age'] > -1) & (df['Transported'] == 1), ['Age', 'Transported']])
df_age_not_transported=pd.DataFrame(df.loc[(df['Age'] > -1) & (df['Transported'] == 0), ['Age', 'Transported']])

ax[0].set(xlabel="Age")
ax[0].set(ylabel="Count")
ax[0].set_title('Transported')
sns.distplot(a=df_age_transported['Age'], label="Transported", kde=False, color='red', bins=10, ax=ax[0])
ax[1].set(xlabel="Age")
ax[1].set(ylabel="Count")
ax[1].set_title('Not Transported')
sns.distplot(a=df_age_not_transported['Age'], label="Not Transported", kde=False, color='blue', bins=10, ax=ax[1])

### How transportation is related with foodcourt?

In [ ]:

fig, ax = plt.subplots(2,1,figsize=(10,20),sharex=False)

df_age_transported=pd.DataFrame(df.loc[(df['FoodCourt'] > -1) & (df['Transported'] == 1), ['FoodCourt', 'Transported']])
df_age_not_transported=pd.DataFrame(df.loc[(df['FoodCourt'] > -1) & (df['Transported'] == 0), ['FoodCourt', 'Transported']])

ax[0].set(xlabel="FoodCourt")
ax[0].set(ylabel="Count")
ax[0].set_title('Transported')
sns.distplot(a=df_age_transported['FoodCourt'], label="Transported", kde=False, color='red', bins=10, ax=ax[0])
ax[1].set(xlabel="FoodCourt")
ax[1].set(ylabel="Count")
ax[1].set_title('Not Transported')
sns.distplot(a=df_age_not_transported['FoodCourt'], label="Not Transported", kde=False, color='blue', bins=10, ax=ax[1])

### How transportation is related with RoomService?

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(10,20),sharex=False)

df_age_transported=pd.DataFrame(df.loc[(df['RoomService'] > -1) & (df['Transported'] == 1), ['RoomService', 'Transported']])
df_age_not_transported=pd.DataFrame(df.loc[(df['RoomService'] > -1) & (df['Transported'] == 0), ['RoomService', 'Transported']])

ax[0].set(xlabel="RoomService")
ax[0].set(ylabel="Count")
ax[0].set_title('Transported')
sns.distplot(a=df_age_transported['RoomService'], label="Transported", kde=False, color='red', bins=10, ax=ax[0])
ax[1].set(xlabel="RoomService")
ax[1].set(ylabel="Count")
ax[1].set_title('Not Transported')
sns.distplot(a=df_age_not_transported['RoomService'], label="Not Transported", kde=False, color='blue', bins=10, ax=ax[1])

### How transportation is related with ShoppingMall?

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(10,20),sharex=False)

df_age_transported=pd.DataFrame(df.loc[(df['ShoppingMall'] > -1) & (df['Transported'] == 1), ['ShoppingMall', 'Transported']])
df_age_not_transported=pd.DataFrame(df.loc[(df['ShoppingMall'] > -1) & (df['Transported'] == 0), ['ShoppingMall', 'Transported']])

ax[0].set(xlabel="ShoppingMall")
ax[0].set(ylabel="Count")
ax[0].set_title('Transported')
sns.distplot(a=df_age_transported['ShoppingMall'], label="Transported", kde=False, color='red', bins=10, ax=ax[0])
ax[1].set(xlabel="ShoppingMall")
ax[1].set(ylabel="Count")
ax[1].set_title('Not Transported')
sns.distplot(a=df_age_not_transported['ShoppingMall'], label="Not Transported", kde=False, color='blue', bins=10, ax=ax[1])

### How transportation is related with Spa?

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(10,20),sharex=False)

df_age_transported=pd.DataFrame(df.loc[(df['Spa'] > -1) & (df['Transported'] == 1), ['Spa', 'Transported']])
df_age_not_transported=pd.DataFrame(df.loc[(df['Spa'] > -1) & (df['Transported'] == 0), ['Spa', 'Transported']])

ax[0].set(xlabel="Spa")
ax[0].set(ylabel="Count")
ax[0].set_title('Transported')
sns.distplot(a=df_age_transported['Spa'], label="Transported", kde=False, color='red', bins=10, ax=ax[0])
ax[1].set(xlabel="Spa")
ax[1].set(ylabel="Count")
ax[1].set_title('Not Transported')
sns.distplot(a=df_age_not_transported['Spa'], label="Not Transported", kde=False, color='blue', bins=10, ax=ax[1])

### How transportation is related with VRDeck?

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(10,20),sharex=False)

df_age_transported=pd.DataFrame(df.loc[(df['VRDeck'] > -1) & (df['Transported'] == 1), ['VRDeck', 'Transported']])
df_age_not_transported=pd.DataFrame(df.loc[(df['VRDeck'] > -1) & (df['Transported'] == 0), ['VRDeck', 'Transported']])

ax[0].set(xlabel="VRDeck")
ax[0].set(ylabel="Count")
ax[0].set_title('Transported')
sns.distplot(a=df_age_transported['VRDeck'], label="Transported", kde=False, color='red', bins=10, ax=ax[0])
ax[1].set(xlabel="VRDeck")
ax[1].set(ylabel="Count")
ax[1].set_title('Not Transported')
sns.distplot(a=df_age_not_transported['VRDeck'], label="Not Transported", kde=False, color='blue', bins=10, ax=ax[1])

## Univariate Analysis

In [ ]:
df_transported=df.loc[df['Transported']==1]
df_not_transported=df.loc[df['Transported']==0]

In [ ]:
df_not_transported

#### VRDeck

In [ ]:
plt.plot(df_transported['VRDeck'],np.zeros_like(df_transported['VRDeck']),'x', color='red')
plt.plot(df_not_transported['VRDeck'],np.zeros_like(df_not_transported['VRDeck']),'o', color='blue')
plt.xlabel('VRDeck')
plt.show()

**Here in this plot the classes are overlapping so it is not possible to seperate using single variable vrdeck**

#### Spa

In [ ]:
plt.plot(df_transported['Spa'],np.zeros_like(df_transported['Spa']),'x', color='red')
plt.plot(df_not_transported['Spa'],np.zeros_like(df_not_transported['Spa']),'o', color='blue')
plt.xlabel('Spa')
plt.show()

**Here in this plot the classes are overlapping so it is not possible to seperate using single variable spa**

#### Shopping Mall

In [ ]:
plt.plot(df_transported['ShoppingMall'],np.zeros_like(df_transported['ShoppingMall']),'x', color='red')
plt.plot(df_not_transported['ShoppingMall'],np.zeros_like(df_not_transported['ShoppingMall']),'o', color='blue')
plt.xlabel('Shopping Mall')
plt.show()

**Here in this plot the some classes are overlapping but still can be used for seperation boundary**

#### Room Service

In [ ]:
plt.plot(df_transported['RoomService'],np.zeros_like(df_transported['RoomService']),'x', color='red')
plt.plot(df_not_transported['RoomService'],np.zeros_like(df_not_transported['RoomService']),'o', color='blue')
plt.xlabel('RoomService')
plt.show()

**Here in this plot the classes are overlapping so it is not possible to seperate using single variable room service**

#### FoodCourt

In [ ]:
plt.plot(df_transported['FoodCourt'],np.zeros_like(df_transported['FoodCourt']),'x', color='red')
plt.plot(df_not_transported['FoodCourt'],np.zeros_like(df_not_transported['FoodCourt']),'o', color='blue')
plt.xlabel('FoodCourt')
plt.show()

**Here in this plot the some classes are overlapping but still can be used for seperation boundary**

#### Age

In [ ]:
plt.plot(df_transported['Age'],np.zeros_like(df_transported['Age']),'x', color='red')
plt.plot(df_not_transported['Age'],np.zeros_like(df_not_transported['Age']),'o', color='blue')
plt.xlabel('Age')
plt.show()

**Here in this plot the classes are overlapping so it is not possible to seperate using single variable room service**

### Here from our analysis only features food court and shopping mall can be helpful for seperation

## Multivariate Analysis

In [ ]:
sns.pairplot(df[['Age','RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Transported']],hue="Transported")

### Duplicate Values

In [ ]:
#check
duplicate = df[df.duplicated()] 
  
print("Duplicate Rows :"+str(len(duplicate)))

In [ ]:
df = df.drop_duplicates()

In [ ]:
#check
duplicate = df[df.duplicated()] 
  
print("Duplicate Rows :"+str(len(duplicate)))

### Encoding

#### Train

In [ ]:
df['VIP'] = df['VIP'].astype('category')
df['HomePlanet'] = df['HomePlanet'].astype('category')
df['CryoSleep'] = df['CryoSleep'].astype('category')
df['Destination'] = df['Destination'].astype('category')
df['Transported'] = df['Transported'].astype('category')

In [ ]:
df['VIP'] = df['VIP'].cat.codes
df['HomePlanet'] = df['HomePlanet'].cat.codes
df['CryoSleep'] = df['CryoSleep'].cat.codes
df['Destination'] = df['Destination'].cat.codes
df['Transported'] = df['Transported'].cat.codes

In [ ]:
df.head()

#### Test

In [ ]:
test['VIP'] = test['VIP'].astype('category')
test['HomePlanet'] = test['HomePlanet'].astype('category')
test['CryoSleep'] = test['CryoSleep'].astype('category')
test['Destination'] = test['Destination'].astype('category')

In [ ]:
test['VIP'] = test['VIP'].cat.codes
test['HomePlanet'] = test['HomePlanet'].cat.codes
test['CryoSleep'] = test['CryoSleep'].cat.codes
test['Destination'] = test['Destination'].cat.codes

In [ ]:
test.head()

### Split X and Y

In [ ]:
# Split the data
# Input/independent variables
features = df.columns[:-1]
X = df[features] # her we are droping the output feature as this is the target and 'X' is input features, the changes are not 
                                # made inplace as we have not used 'inplace = True'
y = df['Transported'] # Output/Dependent variable

In [ ]:
X_test = test[features]

# Feature Enginerring

## Mutual Info

In [ ]:
discrete_features = X.dtypes == int

In [ ]:
mi_scores = mutual_info_classif(X, y, discrete_features=discrete_features)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)

In [ ]:
scores = mi_scores.sort_values(ascending=True)
width = np.arange(len(mi_scores))
ticks = list(mi_scores.index)
plt.barh(width, mi_scores)
plt.yticks(width, ticks)
plt.title("Mutual Information Scores")
plt.figure(dpi=100, figsize=(8, 5))

## Select Kbest

In [ ]:
ordered_rank_features=SelectKBest(score_func=chi2,k=10)
ordered_feature=ordered_rank_features.fit(X,y)

In [ ]:
dfscores=pd.DataFrame(ordered_feature.scores_,columns=["Score"])
dfcolumns=pd.DataFrame(X.columns)

In [ ]:
features_rank=pd.concat([dfcolumns,dfscores],axis=1)

In [ ]:
features_rank.columns=['Features','Score']
features_rank

## DecisionTree Feature Extraction

In [ ]:
model=ExtraTreesClassifier()
model.fit(X,y)

In [ ]:
ranked_features=pd.Series(model.feature_importances_,index=X.columns)
plt.rcParams['figure.figsize'] = [10, 7]
ranked_features.nlargest(25).plot(kind='barh')
plt.show()

## Balancing the count of transported and not transported

In [ ]:
sm = SMOTE(random_state=42)

X_sm, y_sm = sm.fit_resample(X, y)

X_sm=pd.DataFrame(X_sm, columns=X.columns)
y_sm=pd.DataFrame(y_sm, columns=['Transported'])


print('New balance of 1 and 0 classes (%):')
y_sm.value_counts()

# Normalize

## Train

In [ ]:
X_sm = normalize(X_sm, axis=0)

print("Shape of the X :", X_sm.shape)
print("Shape of the y  :", y_sm.shape)

## Test

In [ ]:
X_test = X_test.fillna(X_test.mode().iloc[0])

In [ ]:
X_test = normalize(X_test, axis=0)

# Modelling

### Train Val and Test Split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_sm, y_sm, test_size = 0.3, random_state = 42)

In [ ]:
# lets print the shapes again 
print("Shape of the X Train :", X_train.shape)
print("Shape of the y Train :", y_train.shape)
print("Shape of the X val :", X_val.shape)
print("Shape of the y val :", y_val.shape)

## FLAML and FLU-GUON

In [ ]:
y_train=y_train.to_numpy()

In [ ]:
# Specify automl goal and constraint
automl_settings = {
    "time_budget": 180,  # in seconds
    "metric": 'accuracy',
    "task": 'classification',
    "log_file_name": "spaceship.log",
    "estimator_list": ['lgbm'],
}

In [ ]:
automl.fit(X_train=X_train, y_train=y_train,
           **automl_settings)
clear_output()

In [ ]:
label = 'Transported'
print("Summary of class variable: \n", df[label].describe())

In [ ]:
predictor = TabularPredictor(label=label).fit(df,time_limit=360, presets='best_quality')

# Submission

In [ ]:
preds_valid = automl.predict(X_val)
acc = accuracy_score(y_val,  preds_valid)

In [ ]:
cm = confusion_matrix(y_true=y_val, y_pred=preds_valid)
cm_plot_labels = ['not transported','transported']
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=cm_plot_labels)


print("The val accuracy of the test model is "+str((cm[0][0]+cm[1][1])/y_val.shape[0]))
print("\n")
disp = disp.plot()
plt.show()

In [ ]:
lgb_predictions = 0
test_preds = predictor.predict(test) 
lgb_predictions += test_preds/5

In [ ]:
submission['Transported'] = lgb_predictions.astype("bool")
submission.to_csv("submission.csv",index=False)
submission.head()